In [1]:
import requests
import numpy as np
import pandas as pd
import itertools
import multiprocessing as mp
import datetime
import time
import os
#api_key = '{my_api_key}'
url = 'https://api.polygon.io/v2/ticks/stocks/trades/{symbol}/{date}?timestamp={timestamp}&limit={limit}&apiKey={api_key}'

# Get trade data from polygon.io

In [ ]:
def get_trades(date):
    params = {
    'symbol': 'SPY',
    'date': date,
    'timestamp': '0',
    'limit': 50000,
    'api_key': api_key
    }
    pdata = []
    ct           = params['limit']
    it           = 1
    while ct >= params['limit']:
        response = requests.get(url.format(**params)).json()
        if 'results' in response:
            ct                  = response['results_count']
            pdata              += response['results']
            params['timestamp'] = response['results'][-1]['t']
        else:
            ct = 0
        #print('\r{} | {}...'.format(date, it), end='')
        it += 1
    return pdata

def get_historic_trades(date_start, date_end):
    #get data and write to csv
    dates = [str(x)[:10] for x in pd.date_range(date_start, date_end, freq='D')]
    sd    = 0
    fnum  = 0
    while sd < len(dates):
        with mp.Pool(8) as pool:
            pricing_data = list(itertools.chain.from_iterable(pool.map(get_trades, dates[sd:(sd+100)])))
        print('Exporting {} to {}...'.format(dates[sd], dates[sd+99]))
        spydf = pd.DataFrame(pricing_data)
        spydf.to_csv('SPY{}.csv'.format(fnum), index = False)
        del spydf
        del pricing_data
        sd   += 100
        fnum +=1
    return
    
def export(num):
    #read in trades, export each year to separate file
    print('Reading in SPY{}'.format(num))
    df = pd.read_csv('SPY{}.csv'.format(num))
    print('\tParsing dates')
    df['Date'] = df.t.apply(lambda x: datetime.datetime.fromtimestamp(x/1e9))
    print('\tParsing year')
    df['Year'] = df['Date'].apply(lambda x: x.year)
    print('\tExporting')
    for k, g in df.groupby('Year'):
        print('\t\t{}'.format(k))
        g.to_csv('SPY_{}_{}.csv'.format(k, num), index=False)
    return

In [ ]:
def ohlc(grp):
    d = {'open': grp.p.values[0],
         'high': grp.p.values.max(),
         'low': grp.p.values.min(),
         'close': grp.p.values[-1],
         'volume': grp.s.values.sum()}
    return d

In [3]:
spy = pd.read_csv('./Polygon/Raw/SPY_2003.csv', engine='c')
spy.t = pd.to_datetime(spy.t, unit='ns')
spy['t_microseconds'] = (spy.t.astype(np.int64) // 1e3).astype(np.int64)
spy['t_milliseconds'] = (spy.t.astype(np.int64) // 1e6).astype(np.int64)
spy['t_seconds']      = (spy.t.astype(np.int64) // 1e9).astype(np.int64)
spy

,t,q,i,x,s,c,p,z,e,t_microseconds,t_milliseconds,t_seconds
0,2003-12-01 13:00:25.239,1937025625239119,1937025625239119,11,1000,[12],106.90,1,NaN,1070283625239000,1070283625239,1070283625
1,2003-12-01 13:00:27.739,1937025627739086,1937025627739086,11,200,[12],106.91,1,NaN,1070283627739000,1070283627739,1070283627
2,2003-12-01 13:09:37.222,1937026177222274,1937026177222274,11,500,[12],106.94,1,NaN,1070284177222000,1070284177222,1070284177
3,2003-12-01 13:09:38.722,1937026178722409,1937026178722409,11,1000,[12],106.95,1,NaN,1070284178722000,1070284178722,1070284178
4,2003-12-01 13:20:14.450,1937026814450095,1937026814450095,11,100,[12],106.94,1,NaN,1070284814450000,1070284814450,1070284814
...,...,...,...,...,...,...,...,...,...,...,...,...
353254,2003-12-31 23:08:22.203,1939654102203038,1939654102203038,11,300,[12],111.26,1,NaN,1072912102203000,1072912102203,1072912102
353255,2003-12-31 23:20:01.706,1939654801706930,1939654801706930,11,300,[12],111.26,1,NaN,1072912801706000,1072912801706,1072912801
353256,2003-12-31 23:56:47.143,1939657007143839,1939657007143839,11,100,[12],111.26,1,NaN,1072915007143000,1072915007143,1072915007
353257,2004-01-01 00:44:05.488,1939659845488494,1939659845488494,11,100,[12],111.27,1,NaN,1072917845488000,1072917845488,1072917845


In [ ]:
def agg_ohlcv(df, period):
    

## Once aggregated into OHLCV

In [ ]:
#get trading days (ignores bank holidays and weekends, etc.)
trading_days    = spy['Date'].apply(lambda x: str(x)[:10]).unique()
ndays           = len(trading_days)

#generate list of trading periods (9:30am-4:00pm)
trading_periods = pd.date_range('2000-01-01 09:30:00', '2000-01-01 16:00:00', freq='s').strftime('%H:%M:%S').tolist()

#generate list of trading days, repeat each |trading_period| times
trading_days    = list(itertools.chain.from_iterable(itertools.repeat(tday, len(trading_periods)) for tday in trading_days))

#repeat trading period |ndays| times
trading_periods = list(itertools.chain.from_iterable(itertools.repeat(trading_periods, ndays)))

#merge the two lists into a list of strings, format 'Year-Month-Day Hour:Minute:Seconds'
trading_times   = ['%s %s' % t for t in zip(trading_days, trading_periods)]

In [ ]:
#spy['Datetime'] = spy['Date'].apply(lambda x: str(x)[:19] )

#use the generated trading times to create a new index, and merge original spx pricing data to find missing times
SPY = pd.DataFrame({'Datetime': trading_times})
SPY = SPY.merge(spy, on='Datetime', how='left')
print(SPY.isnull().sum())
SPY

In [ ]:
spy['Date'].values[0][:19]

In [ ]:
spy

In [ ]:
pd.to_datetime('2013-01-02 09:30:00').value

In [ ]:

#for missing data, use Close price of previous minute
SPX['Close'].fillna(method='ffill')
#fill missing data in other columns using close price
SPX['Open'].fillna(SPX['Close'], inplace=True)
SPX['High'].fillna(SPX['Close'], inplace=True)
SPX['Low'].fillna(SPX['Close'], inplace=True)